In [100]:
import pandas as pd
import numpy as np
import tensorflow as tf
y_train_labels = ["toxic",'severe_toxic','obscene','threat','insult','identity_hate']
x_train_label = "comment_text"

dataset_path = "./train_processed.csv"
dataset = pd.read_csv(dataset_path)
dataset.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate,comment_text
0,0000997932d777bf,0,0,0,0,0,0,explanation edits make username hardcore metal...
1,000103f0d9cfb60f,0,0,0,0,0,0,daww match background color im seemingly stick...
2,000113f07ec002fd,0,0,0,0,0,0,hey man im really try edit war guy constantly ...
3,0001b41b1c6bb37e,0,0,0,0,0,0,can not make real suggestion improvement wonde...
4,0001d958c54c6e35,0,0,0,0,0,0,sir hero chance remember page thats


In [101]:
cols = dataset.select_dtypes(include=['object'])
for col in cols.columns.values:
    dataset[col] = dataset[col].fillna('')

In [102]:
x_train = np.array(dataset[[x_train_label]])
y_train = np.array(dataset[y_train_labels])



In [109]:
from rank_bm25 import BM25Okapi

bm25 = BM25Okapi([str(doc).split(" ") for doc in x_train])

def preprocess_function(doc):
    return bm25.get_scores(doc.numpy())

bm25.get_scores(str(x_train[0])).shape

(159571,)

In [104]:
from tensorflow import data

train_dataset = data.Dataset.from_tensor_slices((x_train,y_train))

In [105]:

from keras.models import Sequential
from keras.layers import Dense,LSTM,Bidirectional,Embedding

model = Sequential()

model.add(Embedding(159571+1,32))
model.add(Bidirectional(LSTM(32,activation='relu')))
model.add(Dense(128,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))

model.add(Dense(6,activation='sigmoid'))
model.compile(loss="BinaryCrossentropy",optimizer="adam")



In [106]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, None, 32)          5106304   
                                                                 
 bidirectional_8 (Bidirectio  (None, 64)               16640     
 nal)                                                            
                                                                 
 dense_32 (Dense)            (None, 128)               8320      
                                                                 
 dense_33 (Dense)            (None, 256)               33024     
                                                                 
 dense_34 (Dense)            (None, 128)               32896     
                                                                 
 dense_35 (Dense)            (None, 6)                 774       
                                                      

In [107]:
train_dataset.map(lambda x,y: (tf.py_function(preprocess_function, [x], [tf.float32]),y))
train_dataset = train_dataset.prefetch(data.AUTOTUNE)

In [115]:
test_x = 0
test_y = 0
index = 0
for (x,y) in train_dataset:
    test_y = tf.reshape(y,(1,6))
    test_x = tf.convert_to_tensor([preprocess_function(x)])
    print(test_x)
    print(test_y)
    index+=1
    if index==1:
        break

tf.Tensor([[0. 0. 0. ... 0. 0. 0.]], shape=(1, 159571), dtype=float64)
tf.Tensor([[0 0 0 0 0 0]], shape=(1, 6), dtype=int64)


In [116]:
history = model.fit(test_x,test_y,epochs=1,batch_size=1)